# A Demonstration of the RSA Cryptosystem

The Rivest&ndash;Shamir&ndash;Adleman ([RSA](https://en.wikipedia.org/wiki/RSA_\(cryptosystem\)) cryptosystem demonstrated here combines secure key&ndash;exchange with digital signature to provide for the confidentiality, integrity and authenticity of a private message exchanged between two parties (Alice and Bob) over an insecure channel. Moreover, an adversary (Eve) listening on this channel has no practical way of learning the content of the message, nor any way of tampering with it without the receiving party knowing the message has been corrupted.

The key&ndash;exchange component of this cryptosystem provides for message confidentiality utilizing a [*key&ndash;encapsulation mechanism*](https://en.wikipedia.org/wiki/Key_encapsulation_mechanism) (or *KEM*). This differs from more traditional approaches to key negotiation that utilize a key&ndash;*agreement* mechanism, such as those based on the [discrete logarithm problem](https://en.wikipedia.org/wiki/Discrete_logarithm) introduced by Diffie and Hellman in their seminal 1976 paper [*New Directions in Cryptography*](https://ee.stanford.edu/~hellman/publications/24.pdf).

The digital signature component of this cryptosystem provides for message integrity and authenticity, ideas that were conceived by Diffie and Hellman in their 1976 paper, but not implemented until Rivest, Shamir and Adleman published [*A Method for Obtaining Digital Signatures and Public&ndash;Key Cryptosystems*](https://people.csail.mit.edu/rivest/Rsapaper.pdf) in 1978.

## The Source Code
The following sequence of function calls to the [rsa](https://github.com/dchampion/crypto/tree/master/src/rsa.py) module illustrates a session in which Alice signs and encrypts a private message, and then transmits the message to Bob over an insecure channel. Meanwhile, an adversary (Eve) intercepts the message.

It is assumed that Alice, Bob and Eve all possess their own copy of the [rsa](https://github.com/dchampion/crypto/tree/master/src/rsa.py) module; but that only Alice and Bob possess the signing and encryption keys produced by this module to provide the aforementioned protections.

At the top of each code snippet is a line&ndash;comment to indicate who (i.e., Alice, Bob and/or Eve) is invoking the code. Illustrative code snippets, i.e., those that are not part of the protocol, but rather merely informative, are not commented.

In [1]:
import os
os.chdir("..")

# Alice and Bob (and possibly Eve)
from src import rsa
from tests import sym

Alice and Bob both import the `rsa` module, and a module `sym` that implements a symmetric cipher they have agreed in advance will be used to encrypt and decrypt messages (the `sym` cipher used here is for illustrative purposes only, and in a real-world setting would be replaced with a cryptographically&ndash;strong symmetric cipher such as 3DES or AES).

In [2]:
# Alice
pA, qA, nA, d3A, d5A = rsa.generate_rsa_key(2048)

Alice begins by generating her RSA keys. She does this by calling the `rsa.generate_rsa_key` function, passing it a modulus size of 2048 bits. The function returns the quintuple *pA*, *qA*, *nA*, *d3A* and *d5A* (the suffix *A* on all the parameters identify them as belonging to Alice). Alice transmits her public modulus *nA* to Bob.

Alice must keep the values *pA* and *qA*&mdash;the 1024&ndash;bit prime factors of *nA*&mdash;secret. She must also keep *d3A* and *d5A*&mdash;her signing and decryption keys, respectively&mdash;secret.

To get an idea of the size and proportions of these parameters, let's print some of them out.

In [3]:
print(pA)

127253863270686867916596620460159568142079296931026894309467298337829327913379764941209859229341594554907663676643157733436886006797239832889188273187932517502035902664978845355174299264899161125466305088729130625822226223458493740709177491149453738984887557575242471049558917414898553081498434555182231893217


In [4]:
print(qA)

165867564339343252770565748270866498803732492753327988302144257699691996733583442039578341261155490243199033984952436714506420708120555878527082414355570376572596912277428231017993732615187656552034383900336757452042142960937987721779801502436361883329309622129618726731364881075603327065163764475758377021703


In [5]:
print(nA)

21107288353480643278011343010406949831488302334383261851145674950393659483639096806987696003510084052524962017590124428084011058497222926243959876520522169646470944548779870507990631305751385322693734314882714296233497689109497252329458434023425492958099343324225824630169966886717591868023126651998255284940809709894875563223153609032970651683396020000341504614300214708745406385856485069009868921288303513345456698239535603139338339690281492881965450769789119158834085171325327330030598427859120635257488671734612325791425471892068771260108011128608215057305768976944202424582908670641118583841898693677207587488551


Note that *nA*, which is the product of *pA* and *qA*, is twice the length of each individually; *pA*'s and *qA*'s ~310 digits to *nA*'s ~620.

Recall that Alice requested a modulus of 2048 bits in length in her call to `rsa.generate_rsa_key`. Next, let's print the bit lengths of *nA* and its prime factors.

In [6]:
print(nA.bit_length())

2048


In [7]:
print(pA.bit_length())

1024


In [8]:
print(qA.bit_length())

1024


What we see is that a decimal number of ~620 digits equates to a bit&ndash;length of 2048 (or possibly 2047 owing to implementation details), and further that a decimal number of ~310 digits equates to a bit length of 1024.

Next, we prove that *nA* is in fact the product of *pA* and *qA*.

In [9]:
print(qA * pA == nA)

True


Now it's Bob's turn to generate RSA keys, and transmit his public key *nB* to Alice. To differentiate Bob's keys from Alice's, we append Bob's with the captial letter *B*.

In [10]:
# Bob
pB, qB, nB, d3B, d5B = rsa.generate_rsa_key(2048)

Now that Alice and Bob have generated their keys, and transmitted to each other their public moduli *nA* and *nB*, they can proceed with the secure message exchange.

Let's say Alice wants to send the following message securely to Bob.

In [11]:
# Alice
mA = "Sign and encrypt me!"

First, Alice stores the message "Sign and encrypt me!" in the variable `mA`.

In [12]:
# Alice
oA = rsa.sign(d3A, pA, qA, mA)

Then Alice calls the `rsa.sign` fuction, passing it her secret signing key *d3A*, the two factors of *nA*, *pA* and *qA*, and the message *mA*, to sign the message.

She stores the signature in the variable *oA*.

In [13]:
# Alice
KA, cA = rsa.encrypt_random_key(nB, rsa.ENCRYPTION_EXPONENT)

Next, Alice calls the `rsa.encrypt_random_key` function, passing it Bob's RSA modulus *nB* and the global encryption exponent *rsa.ENCRYPTION_EXPONENT*. This computes a session key that will be used to encrypt the private message using a symmetric cipher (e.g., 3DES, AES). The function returns the pair *KA* and *cA*, which are, respectively, the symmetric encryption (or *session*) key, and the ciphertext of its input material; the same material Bob will use to replicate the session key on his side of the channel.

Alice must keep the session key *KA* private, but she transmits its ciphertext *cA* to Bob.

In [14]:
# Alice
mAC = sym.encrypt(KA, mA)

Next, Alice encrypts the private message, calling the symmetric cipher function `sym.encrypt` and passing it her symmetric key *KA* and the plaintext of her message *mA*.

The result of this operation is the ciphertext *mAC*.

In [15]:
print(mAC)

66997927603696430855694708106000162871261386831697603838128503384771225544638


Here we print the ciphertext to demonstrate that, if Eve were to intercept it, she will see nothing but a seemingly random string of numbers.

Next, Alice transmits the message signature *oA*, the message ciphertext *mAC* and the ciphertext of the symmetric session key *cA* to Bob.

In [16]:
# Bob
KB = rsa.decrypt_random_key(d5B, cA, pB, qB)

In order to decrypt the message, Bob must first recover the session key Alice used to encrypt it. He does this by calling  `rsa.decrypt_random_key`, passing it his private decryption key *d5B*, the ciphertext of the symmetric key that Alice sent to him *cA*, and the two prime factors of his public modulus *pB* and *qB*.

In [17]:
print(KB == KA)

True


Here we prove that the key Bob decrypted above, *KB*, is the same as the key Alice encrypted, *KA*.

In [18]:
# Bob
mB = sym.decrypt(KB, mAC)

Bob calls `sym.decrypt`, passing it *KB* and the message ciphertext *mAC* Alice sent to him, to recover Alice's plaintext message *mA*. He stores the result in *mB*.

In [19]:
print(mB == mA)

True


Here we prove that the recovered plaintext indeed matches Alice's original message.

In [20]:
print(mB)

Sign and encrypt me!


Let's print the message, just to make sure.

In order to complete the session, all that remains is for Bob to verify Alice's signature, and thereby prove that the message has not been altered and, further, that it originated from Alice (or, at least, that it was signed with Alice's private key).

In [21]:
# Bob
print(rsa.verify(nA, rsa.VERIFICATION_EXPONENT, mB, oA))

True


Bob calls the `rsa.verify` fuction, passing Alice's public modulus *nA*, the constant signature&ndash;verification key *rsa.VERIFICATION_EXPONENT*, the decrypted message *mB* and the sigature sent to him by Alice *oA*.

If the assertion above holds, Bob can be assured that the message "Sign and encrypt me!", even if intercepted by Eve, is known only to him and Alice, and that the message was not corrupted in any way by Eve.